<!--<badge>--><a href="https://colab.research.google.com/github/gradsflow/gradsflow/blob/main/examples/nbs/06-AutoModel_fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

In [ ]:
import os

# you can remove this
os.chdir("../../")

In [ ]:
from gradsflow import Model
from gradsflow.tuner.tuner import Tuner
from gradsflow.models.constants import LEARNER

In [ ]:
from timm import create_model
from torch.utils.data import DataLoader
from torchvision.datasets import FakeData

from gradsflow import AutoDataset, Model
from gradsflow.data.image import get_augmentations

image_size = (128, 128)
transform = get_augmentations(image_size)
train_ds = FakeData(size=100, image_size=[3, *image_size], transform=transform)
val_ds = FakeData(size=100, image_size=[3, *image_size], transform=transform)
train_dl = DataLoader(train_ds)
val_dl = DataLoader(val_ds)

num_classes = train_ds.num_classes
autodataset = AutoDataset(train_dl, val_dl, num_classes=num_classes)

cnn1 = create_model("resnet18", pretrained=False, num_classes=num_classes)
cnn2 = create_model("efficientnet_b0", pretrained=False, num_classes=num_classes)

In [ ]:
from gradsflow.tuner.tuner import Tuner
from gradsflow.tuner.automodel import AutoModelV2

In [ ]:
tuner = Tuner()

cnns = tuner.suggest_complex("learner", cnn1, cnn2)
optimizers = tuner.choice("optimizer", "adam", "sgd")
loss = tuner.choice(
    "loss",
    "crossentropyloss",
)

model = AutoModelV2(cnns)

In [ ]:
model.hp_tune(tuner, autodataset, epochs=10)

2021-09-29 18:23:36,147	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265
2021-09-29 18:23:38,658	WARNING function_runner.py:558 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-09-29 18:23:39,886	WARNING worker.py:1215 -- Warning: The actor ImplicitFunc is very large (42 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


Trial name,status,loc,learner,loss,optimizer
_hp_optimizer_444f2_00000,RUNNING,,1,crossentropyloss,adam


(pid=98051) {'learner': <gradsflow.tuner.tuner.ComplexObject object at 0x7f9e76a5e220>}
Result for _hp_optimizer_444f2_00000:
  date: 2021-09-29_18-24-13
  done: false
  experiment_id: 5bbc0bc13f824f7ba3b3aedd98e5e29c
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 1
  node_ip: 192.168.1.84
  pid: 98051
  should_checkpoint: true
  time_since_restore: 28.78166699409485
  time_this_iter_s: 28.78166699409485
  time_total_s: 28.78166699409485
  timestamp: 1632920053
  timesteps_since_restore: 0
  train_loss: 2.5053689443814555
  training_iteration: 1
  trial_id: 444f2_00000
  val_accuracy: 0.14
  val_loss: 2.3365581404929157
  


Trial name,status,loc,learner,loss,optimizer,iter,total time (s),val_loss,val_accuracy,train_loss
_hp_optimizer_444f2_00000,RUNNING,192.168.1.84:98051,1,crossentropyloss,adam,1,28.7817,2.33656,0.14,2.50537


2021-09-29 18:24:36,862	WARNING tune.py:518 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Result for _hp_optimizer_444f2_00000:
  date: 2021-09-29_18-24-43
  done: false
  experiment_id: 5bbc0bc13f824f7ba3b3aedd98e5e29c
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 2
  node_ip: 192.168.1.84
  pid: 98051
  should_checkpoint: true
  time_since_restore: 59.00033497810364
  time_this_iter_s: 30.21866798400879
  time_total_s: 59.00033497810364
  timestamp: 1632920083
  timesteps_since_restore: 0
  train_loss: 2.4123901867625275
  training_iteration: 2
  trial_id: 444f2_00000
  val_accuracy: 0.13
  val_loss: 2.3525213491681405
  


Trial name,status,loc,learner,loss,optimizer,iter,total time (s),val_loss,val_accuracy,train_loss
_hp_optimizer_444f2_00000,RUNNING,192.168.1.84:98051,1,crossentropyloss,adam,2,59.0003,2.35252,0.13,2.41239


Trial name,status,loc,learner,loss,optimizer,iter,total time (s),val_loss,val_accuracy,train_loss
_hp_optimizer_444f2_00000,RUNNING,192.168.1.84:98051,1,crossentropyloss,adam,2,59.0003,2.35252,0.13,2.41239


2021-09-29 18:24:43,544	ERROR tune.py:557 -- Trials did not complete: [_hp_optimizer_444f2_00000]
2021-09-29 18:24:43,546	INFO tune.py:561 -- Total run time: 64.89 seconds (64.27 seconds for the tuning loop).
2021-09-29 18:24:43,547	WARNING tune.py:565 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [ ]:
model.tuner